In [1]:
import tweepy
import csv
import pandas as pd
import sys
import findspark
findspark.init()
import pyspark
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [2]:
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)

In [40]:
spark = SparkSession(sc)

In [3]:
# API credentials here
consumer_key = 'BwfynXQPFAtO7FDB6RshgdJi1'
consumer_secret = 'CfQNAYRO59mpOHnd0E14q04IgrOLNTW18VSXZDFqFi2Y6ql2Ni'
access_token = '2178398821-jLyK8eTM8dTZ0sAy5lrlEy9gzgD8TMyFqQVEcpj'
access_token_secret = 'cmDeuzWdxIRjmcY6FZ6l3CKEk4g3YFlsmyKDu0DOoxZjy'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


In [18]:

# Search word/hashtag value 
HashValue = ""

# search start date value. the search will start from this date to the current date.
StartDate = ""

# getting the search word/hashtag and date range from user
HashValue = input("Enter the hashtag you want the tweets to be downloaded for: ")
StartDate = input("Enter the start date in this format yyyy-mm-dd: ")


Enter the hashtag you want the tweets to be downloaded for: PSL
Enter the start date in this format yyyy-mm-dd: 2021-02-20


In [19]:
tweets = tweepy.Cursor(api.search,q=HashValue,since=StartDate, tweet_mode='extended').items(200)
tweets_data = sc.parallelize(tweets)
tweets_data.take(2)

[Status(_json={'created_at': 'Tue Feb 23 08:14:45 +0000 2021', 'id': 1364126537528737794, 'id_str': '1364126537528737794', 'full_text': 'BEST MEME OF PSL SIX SO FAR! 🤣🤣🤣\n\nhttps://t.co/RdnZYUDC3B', 'truncated': False, 'display_text_range': [0, 57], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 1363920479606632448, 'id_str': '1363920479606632448', 'indices': [34, 57], 'media_url': 'http://pbs.twimg.com/ext_tw_video_thumb/1363920479606632448/pu/img/CIrtMte9rThA7tYa.jpg', 'media_url_https': 'https://pbs.twimg.com/ext_tw_video_thumb/1363920479606632448/pu/img/CIrtMte9rThA7tYa.jpg', 'url': 'https://t.co/RdnZYUDC3B', 'display_url': 'pic.twitter.com/RdnZYUDC3B', 'expanded_url': 'https://twitter.com/fahim_parekh/status/1363920561357811712/video/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 886, 'h': 646, 'resize': 'fit'}, 'small': {'w': 680, 'h': 496, 'resize': 'fit'}, 'medium': {'w': 886, 'h': 64

In [27]:
#extract required information from tweet data
tweet_data_map = tweets_data.map(lambda tweet : [tweet.created_at, tweet.full_text, tweet.user.location.split(','), tweet.lang])
tweet_data_map.take(2)

[[datetime.datetime(2021, 2, 23, 8, 14, 45),
  'BEST MEME OF PSL SIX SO FAR! 🤣🤣🤣\n\nhttps://t.co/RdnZYUDC3B',
  ['Lahore', ' Pakistan'],
  'en'],
 [datetime.datetime(2021, 2, 23, 8, 14, 44),
  'RT @MohsinDotDot: THE MOST ENTERTAINING TEAM OF THE PSL FOR YOU, LAHORE QALANDARS',
  ['Location'],
  'en']]

In [29]:
#filtering valid tweets having country information
valid_tweet = tweet_data_map.filter(lambda tweet: len(tweet[2]) == 2)
tweet_country = valid_tweet.map(lambda tweet: [tweet[0],tweet[1].strip(),tweet[2][1].strip(),tweet[3].strip()])
#filter only English tweets 
twitter_language_filter = tweet_country.filter(lambda tweet: tweet[3] == 'en')
twitter_language_filter.take(2)

[[datetime.datetime(2021, 2, 23, 8, 14, 45),
  'BEST MEME OF PSL SIX SO FAR! 🤣🤣🤣\n\nhttps://t.co/RdnZYUDC3B',
  ['Lahore', ' Pakistan'],
  'en'],
 [datetime.datetime(2021, 2, 23, 8, 14, 13),
  'RT @ESPNcric_stats: Players with 50+ scores in most countries in T20s:\n\n12 - Chris Gayle\n[ENG 7, SA 7, WI 20, NZ 1, IND 37, AUS 7, ZIM 3, B…',
  ['Chennai', 'India'],
  'en']]

In [39]:
tweet_country.take(2)

[[datetime.datetime(2021, 2, 23, 8, 14, 45),
  'BEST MEME OF PSL SIX SO FAR! 🤣🤣🤣\n\nhttps://t.co/RdnZYUDC3B',
  'Pakistan',
  'en'],
 [datetime.datetime(2021, 2, 23, 8, 14, 13),
  'RT @ESPNcric_stats: Players with 50+ scores in most countries in T20s:\n\n12 - Chris Gayle\n[ENG 7, SA 7, WI 20, NZ 1, IND 37, AUS 7, ZIM 3, B…',
  'India',
  'en']]

[[datetime.datetime(2021, 2, 23, 8, 14, 45),
  'BEST MEME OF PSL SIX SO FAR! 🤣🤣🤣\n\nhttps://t.co/RdnZYUDC3B',
  'Pakistan',
  'en'],
 [datetime.datetime(2021, 2, 23, 8, 14, 13),
  'RT @ESPNcric_stats: Players with 50+ scores in most countries in T20s:\n\n12 - Chris Gayle\n[ENG 7, SA 7, WI 20, NZ 1, IND 37, AUS 7, ZIM 3, B…',
  'India',
  'en']]

In [41]:
columns = ["created_at","text", "country", "language"]
df = spark.createDataFrame(twitter_language_filter, columns)
df.show()

+-------------------+--------------------+--------------+--------+
|         created_at|                text|       country|language|
+-------------------+--------------------+--------------+--------+
|2021-02-23 08:14:45|BEST MEME OF PSL ...|      Pakistan|      en|
|2021-02-23 08:14:13|RT @ESPNcric_stat...|         India|      en|
|2021-02-23 08:13:38|@Mjereza231 @JayN...|  South Africa|      en|
|2021-02-23 08:13:29|RT @DaPakistaniLA...|      Pakistan|      en|
|2021-02-23 08:12:59|@darensammy88 Sir...|      Pakistan|      en|
|2021-02-23 08:11:28|@rashidkhan_19 @t...|      Pakistan|      en|
|2021-02-23 08:11:03|@TimGonzalez15 @D...|            PA|      en|
|2021-02-23 08:10:57|RT @Gwadar_Pro: B...|United Kingdom|      en|
|2021-02-23 08:10:30|RT @ESPNcric_stat...|         India|      en|
|2021-02-23 08:09:56|taunted, made fun...|      Pakistan|      en|
|2021-02-23 08:09:27|RT @bhatti_tricep...|      Pakistan|      en|
|2021-02-23 08:08:21|RT @__mahd__rajpu...|      Pakistan|     

In [43]:
#groupBy tweets country bases
df = df.groupBy('country').count().sort(col("count").desc())
df.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|            Pakistan|   19|
|               India|    3|
|        South Africa|    3|
|         Switzerland|    1|
|         Afghanistan|    1|
|                भारत|    1|
|Kingdom of Saudi ...|    1|
|               Japan|    1|
|                  PA|    1|
|                    |    1|
|      United Kingdom|    1|
+--------------------+-----+



In [7]:
for tweet in tweets:
        dfs2.append((tweet.created_at, tweet.full_text.encode('utf-8'), tweet.user.location))
        print(tweet.user.location)
print(dfs2)







Deutschland

Kentinkrono, Kumasi, Ghana 
West Coast
17 scott street, Melrose works
Johannesburg, South Africa
Rising Sun ☀
Australia
Akure, Nigeria
Cape Town, RSA
Ghana
Here, and There 
Lagos, Nigeria
Labone, Accra

Enugu, Nigeria
+256
Haverhill, Suffolk
London
Pretoria, South Africa
London
+256
+256
England, United Kingdom
Manchester, England
औरंगाबाद, भारत
+256

+256
Rhode Island, USA
Cheshire West and Chester
North Sumatra, Indonesia
Darlington, UK
India🇮🇳
IRELAND
+256
Bristol, England
South Africa
Lagos, Nigeria
Lagos, Nigeria
SW🏖Midlands🌈TheShire💛Essex✨
Johannesburg

England, United Kingdom

مسقط, سلطنة عمان
Stroud, England
Gilstead, Yorkshire
Ochil House, Stirling
Kingdom of Saudi Arabia
Delta, Nigeria
Blue Area, Islamabad
USA
Novelda, España

Staffordshire
2812 Noble Fir Court Woodbridg
Bengaluru South, India

Kumasi, Ghana
Abuja, Nigeria
john, vini, lena
ACCRA


Najafgarh, New Delhi
Accra,Ghana

Gurgaon, India


Mumbai, India
England, United Kingdom
Epsom, South East
Curi

KeyboardInterrupt: 

In [9]:
spark = SparkSession(sc)

In [10]:
df = spark.createDataFrame(dfs2, columns)

In [11]:
aaa = df.groupBy("country").count().alias("Tweets Number")
aaa.show()

+--------------------+-----+
|             country|count|
+--------------------+-----+
|        Florida, USA|    1|
|               Abuja|    7|
|      On your device|    1|
|Somewhere Around ...|    1|
|Bengaluru South, ...|    1|
|          Queens, NY|    1|
|    Here, and There |    1|
|Johannesburg, Sou...|    1|
|Ochil House, Stir...|    1|
|       Kumasi, Ghana|    3|
|        Rising Sun ☀|    1|
|      Kaithal, India|    1|
|Volando sobre la ...|    1|
|     Ashaiman, Ghana|    1|
|     Kampala/ Uganda|    1|
|Basingstoke, RG24...|    1|
|  Leicester, England|    2|
|Blue Area, Islamabad|    1|
|       Staffordshire|    1|
|Port Harcourt, La...|    1|
+--------------------+-----+
only showing top 20 rows



In [113]:
aaa.repartition(1).write.csv("tweetsAsPerCountry.csv")

In [12]:
aaa.repartition(1).write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('tweetsAsPerCountry2.csv')